In [ ]:
gene="Cxcl1"

In [ ]:
library(Seurat)
library(ggplot2)
library(tidyverse)

### 读入发育数据

In [ ]:
data1 = readRDS("/data/input/Files/fasica_240722/dev_fascia1_mat_nor_240722.rds")
data2 = readRDS("/data/input/Files/fasica_240722/dev_fascia2_mat_nor_240722.rds")

In [ ]:
df1 = as.data.frame(data1[gene,])
colnames(df1)=c("Exp")
df1$time = as.numeric(rownames(df1))

In [ ]:
df2 = as.data.frame(data2[gene,])
colnames(df2)=c("Exp")
df2$time = as.numeric(rownames(df2))

In [ ]:
df1$group = "fascia1"
df2$group = "fascia2"
df_Dev = rbind(df1,df2)

In [ ]:
head(df_Dev)

### 读入损伤数据

In [ ]:
path = '/data/work/Figure3/gene_plot_data/'
rds = list.files(path=path,pattern = "\\.rds$",full.names=TRUE)
datas = lapply(rds,readRDS)
names(datas) = sub("_myo_trac_mat.rds","",basename(rds))

In [ ]:
dfs = list()
for (age in names(datas)){
print(age)
data = datas[[age]]
df = as.data.frame(data[gene,])
colnames(df)=c("Exp")
df$time = as.numeric(rownames(df))
df$group = age
dfs[[age]]=df
}

In [ ]:
df_Inj = do.call(rbind,dfs)
head(df_Inj)

### 可视化

In [ ]:
df_Dev$time = -df_Dev$time

In [ ]:
head(df_Dev)

In [ ]:
df = rbind(df_Dev,df_Inj)

In [ ]:
color = c("#FFFF00", "#1CE6FF","#108C39" ,"#e61c5d" ,"#D94AE3" ,
              "#FF8B17", "#4463C1")

In [ ]:
names(color) = c("fascia1","fascia2","Adult","P5","E18.5","E17.5","E16.5")

In [ ]:
df$group = factor(df$group,levels=c("fascia1","fascia2","Adult","P5","E18.5","E17.5","E16.5"))


In [ ]:
y_max <- max(quantile(df_Dev$Exp, 0.98),quantile(df_Inj$Exp, 0.98))

In [ ]:
options(repr.plot.width=12,repr.plot.height=6)
p1 <- ggplot(data = df, aes(x = time, y = Exp, color = group)) +
    geom_point(alpha = 0.8,size=0.5) +
  geom_smooth(data = df, aes(x = time, y = Exp, color = group),method = "loess", se = FALSE,size=1.5)+
    theme_bw() +
    scale_y_continuous(limits = c(0, y_max), oob = scales::oob_squish) +  # 设置 Y 轴范围
    labs(title = gene) +
  theme(  
    axis.text.x = element_blank(),  # 隐藏 x 轴坐标文本
    axis.ticks.x = element_blank(),
    plot.title = element_text(hjust = 0.5, size = 20, face = "bold", color = "white"), 
    axis.text.y = element_text(size = 12, color = "white"), 
    plot.background = element_rect(fill = "black"), 
    panel.background = element_rect(fill = "black"), 
    panel.grid.major = element_blank(), 
    panel.grid.minor = element_blank(),
    axis.line = element_line(color = "white"),  # 设置坐标轴颜色为白色
    axis.ticks = element_line(color = "white")
  )+
  scale_color_manual(values=color)+
  scale_x_continuous(sec.axis = dup_axis()) +  # 添加辅助 x 轴
  geom_vline(xintercept = 0, color = "white", size = 0.5)
p1

In [ ]:
ggsave(plot=p1,paste0("/data/work/Figure3/",gene,"_Exp_plot.pdf"),w=12,h=6)